In [1]:
!pip install pandas tqdm datasets torch torchtext transformers sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is

In [2]:
from datasets import load_dataset, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

# Load Data

In [3]:
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Diseases_Symptoms.csv:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [4]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [5]:
updated_data = [{"Name":item["Name"], "Symptoms":item["Symptoms"]} for item in data_sample["train"]]

In [6]:
df = pd.DataFrame(updated_data)

In [8]:
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [9]:
df["Symptoms"] = df["Symptoms"].apply(lambda x: ", ".join(x.split(", ")))

In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [12]:
device

device(type='cuda')

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained("distilbert/distilgpt2")
model = GPT2LMHeadModel.from_pretrained("distilbert/distilgpt2").to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [24]:
BATCH_SIZE = 8

In [25]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [26]:
class LanguageDataset(Dataset):
  def __init__(self, data, tokenizer):
    self.labels = df.columns
    self.data = df.to_dict(orient="records")
    self.tokenizer = tokenizer
    x = self.fittest_max_length(df)
    self.max_length = x

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    x = self.data[idx][self.labels[0]]
    y = self.data[idx][self.labels[1]]
    text = f"{x} | {y}"
    tokens = self.tokenizer.encode_plus(text, return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True)
    return tokens

  def fittest_max_length(self, df):
    max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
    x=2
    while x < max_length: x = x * 2
    return x

In [27]:
data_sample = LanguageDataset(df, tokenizer)

In [28]:
data_sample

In [29]:
train_size = int(0.8 * len(data_sample))
val_size = len(data_sample) - train_size
train_data, val_size = random_split(data_sample, [train_size, val_size])

In [30]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_size, batch_size=BATCH_SIZE)

In [31]:
num_epochs = 10

In [32]:
batch_size = BATCH_SIZE
model_name = "distilbert/distilgpt2"
gpu = 0

In [33]:
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [34]:
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu', "train_loss", "validation_loss", "epoch_duration_sec"])

In [35]:
results

,epoch,transformer,batch_size,gpu,train_loss,validation_loss,epoch_duration_sec


In [36]:
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    valid_iterator = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(val_loader)

    end_time = time.time()
    epoch_duration_sec = end_time - start_time

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(val_loader)}")

Training Epoch 1/10 Batch Size: 8, Transformer: distilbert/distilgpt2: 100%|██████████| 40/40 [00:33<00:00,  1.20it/s, Training Loss=0.236]
Validation Epoch 1/10: 100%|██████████| 10/10 [00:02<00:00,  4.32it/s, Validation Loss=0.201]


Epoch: 1, Validation Loss: 0.21136091649532318


Training Epoch 2/10 Batch Size: 8, Transformer: distilbert/distilgpt2: 100%|██████████| 40/40 [00:32<00:00,  1.23it/s, Training Loss=0.184]
Validation Epoch 2/10: 100%|██████████| 10/10 [00:02<00:00,  4.10it/s, Validation Loss=0.163]


Epoch: 2, Validation Loss: 0.17087869346141815


Training Epoch 3/10 Batch Size: 8, Transformer: distilbert/distilgpt2: 100%|██████████| 40/40 [00:33<00:00,  1.21it/s, Training Loss=0.159]
Validation Epoch 3/10: 100%|██████████| 10/10 [00:02<00:00,  4.29it/s, Validation Loss=0.162]


...

Epoch: 8, Validation Loss: 0.2176605761051178


Training Epoch 9/10 Batch Size: 8, Transformer: distilbert/distilgpt2: 100%|██████████| 40/40 [00:32<00:00,  1.23it/s, Training Loss=0.0485]
Validation Epoch 9/10: 100%|██████████| 10/10 [00:02<00:00,  4.17it/s, Validation Loss=0.221]


Epoch: 9, Validation Loss: 0.21988753974437714


Training Epoch 10/10 Batch Size: 8, Transformer: distilbert/distilgpt2: 100%|██████████| 40/40 [00:32<00:00,  1.22it/s, Training Loss=0.0213]
Validation Epoch 10/10: 100%|██████████| 10/10 [00:02<00:00,  4.23it/s, Validation Loss=0.239]

Epoch: 10, Validation Loss: 0.23778212070465088


In [37]:
input_str = "Kidney Failure"

In [39]:
input_ids = tokenizer.encode(input_str, return_tensors="pt").to(device)

In [40]:
input_ids

tensor([[48374,  1681, 25743]], device='cuda:0')

In [41]:
output = model.generate(input_ids, max_length=20, num_return_sequences = 1,
                        do_sample=True, top_k=8, top_p=0.95, temperature=0.5, repetition_penalty=1.2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [42]:
output

tensor([[48374,  1681, 25743,   930,  1001,  4119, 32692,  2356,    11, 29844,
           393,  1067,  9430,    11,  2910,   287, 18922,    11, 10792,  2956]],
       device='cuda:0')

In [43]:
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

In [44]:
decoded_output

'Kidney Failure | Severe abdominal pain, swelling or cramps, blood in urine, frequent ur'

In [45]:
torch.save(model, "SLM_Fine-Tuning.pt")